# Análise de Sentimentos de comentários no Twitter | Transformações.


**Giovano Panatta**

Uma stopword é uma palavra frequentemente removida de textos no processamento de linguagem natural por ser comum e adicionar pouco significado. Exemplos em português incluem: e, o, a, de, que.

**Vamos treinar um modelo de análise de sentimentos usando o conjunto de dados do Twitter, aplicando a técnica de bag-of-words. O objetivo será analisar o sentimento expresso pelos usuários no conjunto de validação. Escolheremos um modelo de aprendizado de máquina adequado para esta tarefa.**

In [34]:
# Importando as bibliotecas

import pandas as pd
import json

# Importando a biblioteca para expressões regulares (regex)
import re

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, PowerTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import classification_report



In [35]:
# Carregamento ddos dados
train_df = pd.read_csv('twitter_training.csv', header=None) #Não contém nomes nas colunas
validation_df = pd.read_csv('twitter_validation.csv', header=None)

In [36]:
train_df.head()

,0,1,2,3
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [37]:
validation_df.head()

,0,1,2,3
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [38]:
#Verificando a estrutura dos dados
train_df.shape, validation_df.shape

((74682, 4), (1000, 4))

In [39]:
#Verificando se há dados nulos em train_df
train_df.isnull().sum()

0      0
1      0
2      0
3    686
dtype: int64

In [40]:
#Verificando se há dados nulos em validation_df
validation_df.isnull().sum()

0    0
1    0
2    0
3    0
dtype: int64

In [41]:
#Excluindo valores nulos de train_df
train_df.dropna(inplace=True)
train_df.isnull().sum()


0    0
1    0
2    0
3    0
dtype: int64

**Obs:** Devido a natureza dos nossos dados faltantes (texto), seria difícil tratá-los com métodos estatísticos comuns. E, lavando em consideração que possuímos uma quantidade substancial de dados válidos, a exclusão dos faltantes pareceu ser um caminho válido>

In [42]:
# Verificando se há linhas duplicadas
duplicates_train = train_df.duplicated().any()


duplicates_validation = validation_df.duplicated().any()

print("Linhas duplicadas no train_df:", duplicates_train)
print("Linhas duplicadas no validation_df:", duplicates_validation)


Linhas duplicadas no train_df: True
Linhas duplicadas no validation_df: False


In [43]:
#Verificando duplicadas
num_duplicates_train = train_df.duplicated().sum()
num_duplicates_train

2340

**Obs:** Conforme podemos observar, em nosso dataset de treino, além de entradas NaN ainda contamos com 2340 linhas duplicadas. Desta forma, para que consigamos resulados mais realistas em nossos modelos de ML, teremos também que excluir esses dados:

In [44]:
#Exluindo linhas duplicadas
train_df.drop_duplicates(inplace=True)
duplicates_train = train_df.duplicated().any()
duplicates_train


False

In [45]:
#Verificando novamente a estrutura do DF
train_df.shape, validation_df.shape

((71656, 4), (1000, 4))

In [46]:
# Função para pré-processar os textos
def preprocess_text(text):
    # com regex: substituindo URL's por strings vazias
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE) # ^ passa a corresponder também ao início de cada linha dentro da string.
                                                                            # $ passa a corresponder ao fim de cada linha.
    # Removendo caracteres especiais e números
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)
    # Removendo espaços extras
    text = re.sub(r'\s+', ' ', text, flags=re.I) #faz com que a expressão regular ignore a diferença entre letras maiúsculas e minúsculas
    # Convertendo para Minúsculas
    text = text.lower()
    return text

# Aplicando o prép rocessamento na coluna 3 do dataset
train_df[3] = train_df[3].astype(str).apply(preprocess_text)
validation_df[3] = validation_df[3].astype(str).apply(preprocess_text)

# Convertendo os textos dos tweets em uma matriz bag-of-words com stop_words em inglês
vectorizer = CountVectorizer(stop_words='english')
X_train_counts = vectorizer.fit_transform(train_df[3])
X_validation_counts = vectorizer.transform(validation_df[3])

# Preparando as variáveis alvo sendo a coluna 2 (irrelevant, negative, neutral ou positive)
y_train = train_df[2]
y_validation = validation_df[2]

# Inicializando o modelo de Regressão Logística com 1000 iterações (outros testes realizados, este performou melhor)
log_reg_classifier = LogisticRegression(max_iter=1000, tol=0.1)

# Treinando o modelo
log_reg_classifier.fit(X_train_counts, y_train)

# Avaliando o modelo
y_pred_log_reg = log_reg_classifier.predict(X_validation_counts)
print(classification_report(y_validation, y_pred_log_reg))

              precision    recall  f1-score   support

  Irrelevant       0.95      0.94      0.94       172
    Negative       0.93      0.95      0.94       266
     Neutral       0.97      0.95      0.96       285
    Positive       0.94      0.94      0.94       277

    accuracy                           0.95      1000
   macro avg       0.95      0.95      0.95      1000
weighted avg       0.95      0.95      0.95      1000



**Vamos usar o mesmo conjunto de dados do Twitter da questão anterior, mas agora aplicaremos a técnica bag-of-ngrams para treinar um modelo de análise de sentimentos. Compararemos o desempenho de n-grams com duas e três palavras para ver qual oferece melhores insights sobre os sentimentos dos usuários. Esse processo nos ajudará a entender o impacto do tamanho dos n-grams na precisão da análise de sentimentos.**

In [47]:
# Configurando Count vectorizer para bigrams (2,2)
vectorizer_bigrams = CountVectorizer(stop_words='english', ngram_range=(2, 2))
X_train_bigrams = vectorizer_bigrams.fit_transform(train_df[3])
X_validation_bigrams = vectorizer_bigrams.transform(validation_df[3])

# Treinando o modelo de Regressão Logística com bigrams com tolerância de convergência 0.1
log_reg_classifier_bigrams = LogisticRegression(max_iter=1000, tol=0.1)
log_reg_classifier_bigrams.fit(X_train_bigrams, y_train)

# Avaliando o modelo no conjunto de validação com bigrams
y_pred_bigrams = log_reg_classifier_bigrams.predict(X_validation_bigrams)
report_bigrams = classification_report(y_validation, y_pred_bigrams)

# Configurando Count vectorizer para trigrams (3,3)
vectorizer_trigrams = CountVectorizer(stop_words='english', ngram_range=(3, 3))
X_train_trigrams = vectorizer_trigrams.fit_transform(train_df[3])
X_validation_trigrams = vectorizer_trigrams.transform(validation_df[3])

# Treinando o modelo de Regressão Logistica com trigrams com tolerância de convergência 0.1
log_reg_classifier_trigrams = LogisticRegression(max_iter=1000, tol=0.1)
log_reg_classifier_trigrams.fit(X_train_trigrams, y_train)

# Avaliando o modelo no conjunto de validação com trigrams
y_pred_trigrams = log_reg_classifier_trigrams.predict(X_validation_trigrams)
report_trigrams = classification_report(y_validation, y_pred_trigrams)

report_bigrams_json = classification_report(y_validation, y_pred_bigrams, output_dict=True)
report_trigrams_json = classification_report(y_validation, y_pred_trigrams, output_dict=True)

#Transformando o report em JSON para organizar a visualização
df_bigrams = pd.DataFrame(report_bigrams_json).transpose()
df_trigrams = pd.DataFrame(report_trigrams_json).transpose()


print("Bigrams Report:")
print(df_bigrams)
print("\nTrigrams Report:")
print(df_trigrams)


Bigrams Report:
              precision    recall  f1-score  support
Irrelevant     1.000000  0.976744  0.988235   172.00
Negative       0.980695  0.954887  0.967619   266.00
Neutral        0.992780  0.964912  0.978648   285.00
Positive       0.922297  0.985560  0.952880   277.00
accuracy       0.970000  0.970000  0.970000     0.97
macro avg      0.973943  0.970526  0.971845  1000.00
weighted avg   0.971283  0.970000  0.970225  1000.00

Trigrams Report:
              precision    recall  f1-score   support
Irrelevant     1.000000  0.918605  0.957576   172.000
Negative       0.915385  0.894737  0.904943   266.000
Neutral        1.000000  0.929825  0.963636   285.000
Positive       0.842271  0.963899  0.898990   277.000
accuracy       0.928000  0.928000  0.928000     0.928
macro avg      0.939414  0.926766  0.931286  1000.000
weighted avg   0.933801  0.928000  0.929074  1000.000



* Modelo com Bigrams: Acurácia de 97%, resultados consistentes e precisos para todas as classes.

* Modelo com Trigrams: Acurácia de 93%, resultados ligeiramente inferiores

Conclusão: Bigrams superaram os trigrams, oferecendo resultados mais consistentes e precisos para análise de sentimentos do Twitter.

**Vamos aplicar a técnica TF-IDF ao mesmo conjunto de dados do Twitter para treinar um modelo de análise de sentimentos. O objetivo é verificar e comparar o desempenho desta abordagem com as técnicas anteriores, bag-of-words e bag-of-ngrams. A técnica TF-IDF nos ajudará a entender como a frequência de palavras e a importância delas no contexto do conjunto de dados influenciam a análise dos sentimentos expressos pelos usuários.**

In [48]:
# Configurando o TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = tfidf_vectorizer.fit_transform(train_df[3].astype(str)) # Convertendo para string para garantir compatibilidade
X_validation_tfidf = tfidf_vectorizer.transform(validation_df[3].astype(str))

# Treinando o modelo de Regressão Logística com 1000 iterações e tolerância de convergência 0.1
log_reg_classifier_tfidf = LogisticRegression(max_iter=1000, tol=0.1) 
log_reg_classifier_tfidf.fit(X_train_tfidf, y_train)

# Avaliando o modelo com o conjunto de validação ()
y_pred_tfidf = log_reg_classifier_tfidf.predict(X_validation_tfidf)

# Gerando e formatando o relatório em DF para facilitar a visualização
report_dict = classification_report(y_validation, y_pred_tfidf, output_dict=True)
df_report = pd.DataFrame(report_dict).transpose()

# Imprimindo o DataFrame para visualização clara
print(df_report)


              precision    recall  f1-score   support
Irrelevant     0.916667  0.895349  0.905882   172.000
Negative       0.881119  0.947368  0.913043   266.000
Neutral        0.951673  0.898246  0.924188   285.000
Positive       0.916968  0.916968  0.916968   277.000
accuracy       0.916000  0.916000  0.916000     0.916
macro avg      0.916606  0.914483  0.915020  1000.000
weighted avg   0.917271  0.916000  0.916075  1000.000


**Conclusão**: Com a acurácia, o modelo previu corretamente a classificação de 91.6% das amostras.

**Vamos trabalhar com o dataset Breast Cancer Wisconsin, disponível através do sklearn.datasets.load_breast_cancer. O processo incluirá a transformação dos dados utilizando técnicas como MinMaxScaler e StandardScaler. Além disso, aplicaremos pelo menos mais duas técnicas de transformação. Com os dados transformados, utilizaremos um modelo de regressão logística para avaliar o desempenho em termos de precisão, recall, e outras métricas relevantes.**

**Este processo nos ajudará a entender como diferentes técnicas de pré-processamento de dados podem influenciar a performance de modelos de aprendizado de máquina, especialmente em tarefas de classificação como a detecção de câncer de mama.**

In [49]:
# Carregando o conjunto.
data = load_breast_cancer()

# Criando o dataframe com as colunas como features
df = pd.DataFrame(data.data, columns=data.feature_names)

# Adicionando a variável alvo 'target' ao DF
df['target'] = data.target


print(df.head())

   mean radius  mean texture  mean perimeter  mean area  mean smoothness  \
0        17.99         10.38          122.80     1001.0          0.11840   
1        20.57         17.77          132.90     1326.0          0.08474   
2        19.69         21.25          130.00     1203.0          0.10960   
3        11.42         20.38           77.58      386.1          0.14250   
4        20.29         14.34          135.10     1297.0          0.10030   

   mean compactness  mean concavity  mean concave points  mean symmetry  \
0           0.27760          0.3001              0.14710         0.2419   
1           0.07864          0.0869              0.07017         0.1812   
2           0.15990          0.1974              0.12790         0.2069   
3           0.28390          0.2414              0.10520         0.2597   
4           0.13280          0.1980              0.10430         0.1809   

   mean fractal dimension  ...  worst texture  worst perimeter  worst area  \
0             

In [50]:
df.shape

(569, 31)

In [51]:
df.isnull().sum()

mean radius                0
mean texture               0
mean perimeter             0
mean area                  0
mean smoothness            0
mean compactness           0
mean concavity             0
mean concave points        0
mean symmetry              0
mean fractal dimension     0
radius error               0
texture error              0
perimeter error            0
area error                 0
smoothness error           0
compactness error          0
concavity error            0
concave points error       0
symmetry error             0
fractal dimension error    0
worst radius               0
worst texture              0
worst perimeter            0
worst area                 0
worst smoothness           0
worst compactness          0
worst concavity            0
worst concave points       0
worst symmetry             0
worst fractal dimension    0
target                     0
dtype: int64

In [52]:
duplicated = df.duplicated().any()
duplicated

False

In [53]:
#Atribuindo os dados a X features e y 'target'
X = df.drop('target', axis=1)
y = df['target']

# Dividindo o dataset em conjuntos de treino 70% e teste 30%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Iniciando as transformações (aplicando em um dicionário)
scalers = {
    "MinMaxScaler": MinMaxScaler(),
    "StandardScaler": StandardScaler(),
    "RobustScaler": RobustScaler(),
    "PowerTransformer": PowerTransformer()
}

# Avaliando o modelo com cada transformador
results = {}
#scaler name = a chave do dicionário de transformadores, scaler = o valor
for scaler_name, scaler in scalers.items(): #utilizando o dicionário (tuplas) que contem as transformações
    # Aplicando a transformação
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Treinando o modelo de regressão logística com 1000 iterações
    model = LogisticRegression(max_iter=10000)
    model.fit(X_train_scaled, y_train)
    
    # Avaliando o modelo
    y_pred = model.predict(X_test_scaled)
    report = classification_report(y_test, y_pred, output_dict=True) #output_dict=True (retorna como dicionário)
    results[scaler_name] = report['accuracy']

results


{'MinMaxScaler': 0.9649122807017544,
 'StandardScaler': 0.9824561403508771,
 'RobustScaler': 0.9883040935672515,
 'PowerTransformer': 0.9824561403508771}

**Conclusão:** As técnicas de transformação de dados melhoraram a acurácia do modelo de regressão logística no dataset Breast Cancer Wisconsin, com o RobustScaler alcançando a melhor acurácia (98.83%), mostrando a eficácia de adaptar a transformação às características do dataset, especialmente na presença de outliers.